In [1]:
a = [1.0,2.0,1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0

In [4]:
a

[1.0, 2.0, 3.0]

In [5]:
import torch


In [17]:
a = torch.ones(3)   # 1 维
a

tensor([1., 1., 1.])

In [18]:
float(a[1])

1.0

In [20]:
a[1]

tensor(1.)

In [21]:
# 将list传给tensor，变为tensor
points = torch.tensor([4.0,1.0,5.0,3.0,2.0,1.0])   # 把坐标存在1D tensor
points

tensor([4., 1., 5., 3., 2., 1.])

In [24]:
float(points[0]),float(points[1])

(4.0, 1.0)

In [25]:
points = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [29]:
points.size()

torch.Size([3, 2])

In [ ]:
# named tensors

In [34]:
img_t = torch.randn(3,5,5)  #模仿图像的维度
weights = torch.tensor([0.2126,0.7152,0.0722])   # 每个通道的权重
# 构建有两个batch的tensor
batch_t = torch.randn(2,3,5,5)  

In [35]:
img_gray_naive = img_t.mean(-3)   # 通道维度上未加权的平均 即转换为灰度图像
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape,batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [36]:
unsqueezed_weights= weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t*unsqueezed_weights)
batch_weights = (batch_t*unsqueezed_weights)    #将每个通道里的像素都与权重相乘
img_gray_weighted = img_weights.sum(-3)        # 三通道变为单通道
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape,batch_t.shape,unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [37]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw',img_t,weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw',batch_t,weights) #以上操作的快捷方法 
batch_gray_weighted_fancy.shape   

torch.Size([2, 5, 5])

In [38]:
weights_named = torch.tensor([0.2126,0.7152,0.0722],names=['channels'])
weights_named

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [44]:
img_named =img_t.refine_names(...,'channels','rows','columns')
batch_named = batch_t.refine_names(...,'channels','rows','columns')

print('img named:',img_named.shape,img_named.names)
print('batch named:',batch_named.shape,batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [45]:
weights_aligned = weights_named.align_as(img_named)   # 自动补齐维度
weights_aligned.shape,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [46]:
gray_named = (img_named*weights_aligned).sum('channels')  # 一些函数也接受命名的维度作为参数
gray_named.shape,gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [47]:
# 取消维度命名

gray_plain = gray_named.rename(None)
gray_plain.shape,gray_plain.names

(torch.Size([5, 5]), (None, None))

In [49]:
# 指定张量的类型,使用参数

double_points = torch.ones(10,2,dtype = torch.double)
short_points = torch.tensor([[1,2],[3,4]],dtype = torch.short)

In [50]:
short_points.dtype


torch.int16

In [51]:
# 使用函数转换数值类型

double_points = torch.zeros(10,2).double()
short_points=torch.ones(10,2).short()

In [52]:
# 或者
double_points=torch.zeros(10,2).to(torch.double)
short_points = torch.ones(10,2).to(dtype=torch.short)

In [ ]:
# tensor API


In [62]:
a = torch.ones(3,2,5)
a_t = torch.transpose(a,0,2)    #指定维度的转置
a.shape,a_t.shape

(torch.Size([3, 2, 5]), torch.Size([5, 2, 3]))

In [64]:
a_t2 = a.transpose(1,2)
a_t2.shape

torch.Size([3, 5, 2])

### tensor 操作的分类

- 构建操作：构造张量的函数，比如 *ones* 和 *from_numpy*
- 索引、切片、连接、变形操作：改变shape,步长等函数，比如 *transpose*
- 数学操作
    
 

In [71]:
# 在存储的角度看张量

points =torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points.storage()   # 尽管张量是3行2列，但是被存储在一个连续的数组中，也就是说，存储总是1维的

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [72]:
points_storage = points.storage()   # 将points_storage 指向存储的points
points_storage[0]            # 手动索引

4.0

In [73]:
points_storage[0] = 11.0     #改变存储的值自然会改变tensor的值
points

tensor([[11.,  1.],
        [ 5.,  3.],
        [ 2.,  1.]])

In [75]:
points[2,1] = 50.0        # 改变tensor的值也会改变存储的值
points.storage() 

 11.0
 1.0
 5.0
 3.0
 2.0
 50.0
[torch.FloatStorage of size 6]

In [80]:
# 就地操作如   zeros_()  并不是新建一个tensor，而是在原地操作，改变原张量
# 任何不带下划线的方法都不会改变源张量，而是返回一个新的张量:


a = torch.ones(3, 2)

a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [78]:
a.storage()

 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
[torch.FloatStorage of size 6]

### Tensor metadata: Size, offset, and stride


![](https://raw.githubusercontent.com/liuzhaoo/markdown_pics/master/img/size,offset,stride.png)